In [1]:
from utils import time_op

In [2]:
# tests the model counting capabilities of SharpSAT-TD and Buddy
from formulas import GPMC, Formula, BuddyNode, BuddyContext, random_k_cnf

n,m,k = 8, 10, 7
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx}" for idx in range(1, n+1)]

solver = GPMC()
satcount_gpmc = time_op(solver.satcount)(cnf)
print(f"GPMC satcount = {satcount_gpmc}")

with BuddyContext(vars) as ctx:
    f = time_op(ctx.parse)(formula)
    satcount_bdd = time_op(f.satcount)()
    print(f"BDD satcount = {satcount_bdd}")


[00013.3257 ms / 0.0133 s / 0.0002 min] satcount
GPMC satcount = 224
[00079.7815 ms / 0.0798 s / 0.0013 min] parse
[00000.1211 ms / 0.0001 s / 0.0000 min] satcount
BDD satcount = 224.0


In [3]:
n,clauses_to_variables,k = 20, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]

In [4]:
import impmeas.bdds as bdd 
from formulas import BuddyContext

with BuddyContext(vars) as ctx:
    x = "x1"
    f = time_op(ctx.parse)(formula)
    print(f"bdd size = {f.nodecount} with satcount = {f.satcount()}")
    blame, ub = time_op(bdd.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(bdd.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[01723.4070 ms / 1.7234 s / 0.0287 min] parse
bdd size = 9820 with satcount = 473057.0
=== COMPUTING BLAME for x1 in BDD with size 9820 ===
k=0 size ell=4785 d result=0.1354 max increase possible=0.4323 
k=1 size ell=28895 d result=0.1578 max increase possible=0.1830 
k=2 size ell=27894 d result=0.1319 max increase possible=0.0383 
k=3 size ell=5889 d result=0.0357 max increase possible=0.0021 
k=4 size ell=241 d result=0.0021 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.4629, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[00401.9504 ms / 0.4020 s / 0.0067 min] blame
[00000.2589 ms / 0.0003 s / 0.0000 min] influence
influence 0.13536, 0.46287 <= blame <= 0.46288, blame diff 0.00001


In [5]:
import impmeas.mc as mc
from formulas import GPMC, Formula, FormulaContext

solver = GPMC(cs=20000)
ctx = FormulaContext(solver)
f = time_op(ctx.parse)(formula)
x = "x1"
infl = (time_op)(mc.influence)(f, x)
blame, ub = time_op(mc.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")

[01880.4331 ms / 1.8804 s / 0.0313 min] parse
[00368.2878 ms / 0.3683 s / 0.0061 min] influence
=== COMPUTING BLAME for x1 in Formula with size 3830 ===
k=0 size of cnf: 3842 d result=0.1354 max increase possible=0.4323 
k=1 size of cnf: 3913 d result=0.1578 max increase possible=0.1830 
k=2 size of cnf: 3980 d result=0.1319 max increase possible=0.0383 
k=3 size of cnf: 4043 d result=0.0357 max increase possible=0.0021 
k=4 size of cnf: 4102 d result=0.0021 max increase possible=0.0000 
stopped earlier because cannot improve above cutoff.
current value: 0.4629, can be increased by 0.0000 <= 0.0001.
=== DONE ===
[161820.4625 ms / 161.8205 s / 2.6970 min] blame
influence 0.13536, 0.46287 <= blame <= 0.46288, blame diff 0.00001


In [6]:
import importance_measures.mc as mc 
from formulas import Formula, GPMC

# f,S = mc.at_most(2, {"x1","x2","x3", "x4"})
f,S = mc.at_most_cnf(3, {1,2,3,4}, 5)
print(f)
GPMC().satcount( f )

[[7, -1], [-7, 1], [2, 7, -5], [-2, 5], [-7, 5], [3, 5, -11], [-3, 11], [-5, 11], [4, 11, -9], [-4, 9], [-11, 9], [8, -2, -7], [-8, 7], [-8, 2], [-6, 5, 8], [-6, 3, 8], [6, -3, -5], [6, -8], [-14, 11, 6], [-14, 4, 6], [14, -4, -11], [14, -6], [13, -3, -8], [-13, 8], [-13, 3], [-10, 6, 13], [-10, 4, 13], [10, -4, -6], [10, -13], [12, -4, -13], [-12, 13], [-12, 4], [-12]]


15

In [7]:
from re import A
from utils import time_op

import impmeas.bdds as bdd 
import impmeas.table as table
from formulas import BuddyContext, TableContext, random_k_cnf

n,clauses_to_variables,k = 10, 4.5, 7
m = int(n*clauses_to_variables)
cnf, formula = random_k_cnf(n,m,k)
vars = [f"x{idx+1}" for idx in range(n)]
x = "x1"
formula = "x1 | (x2 ^ x3)"

ctx = TableContext(print_mode="primes")
f = time_op(ctx.parse)(formula)
blame, ub = time_op(table.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
print(f"{blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")
infl = time_op(table.influence)(f, x)
print(f"influence = {infl:.5f}")
hammer = time_op(table.bz_hcgm)(f, x)
print(f"hammer = {hammer:.5f}")
omega = time_op(table.omega)(f)
upsilon = time_op(table.upsilon)(f)

print("="*100)

with BuddyContext(vars) as ctx:
    f = time_op(ctx.parse)(formula)
    print(f"bdd size = {f.nodecount} with satcount = {f.satcount()}")
    blame, ub = time_op(bdd.blame)(f, x, rho=lambda x: 1/(x+1), cutoff=1e-4, debug=True)
    infl = time_op(bdd.influence)(f, x)
    print(f"influence {infl:.5f}, {blame:.5f} <= blame <= {ub:.5f}, blame diff {ub-blame:.5f}")


[00002.1629 ms / 0.0022 s / 0.0000 min] parse
=== COMPUTING BLAME for x1 in Table with size 8 ===
k=0 d result=0.5000 max increase possible=0.2500 
k=1 d result=0.1250 max increase possible=0.0833 
k=2 
stopped earlier because no change in g(f,0,k) and g(f,1,k) occurred.
=== DONE ===
[00001.2770 ms / 0.0013 s / 0.0000 min] blame
0.62500 <= blame <= 0.62500, blame diff 0.00000
[00000.0587 ms / 0.0001 s / 0.0000 min] influence
influence = 0.50000
[00000.2902 ms / 0.0003 s / 0.0000 min] bz_hcgm
hammer = 0.31250
[00000.6571 ms / 0.0007 s / 0.0000 min] omega
[00000.6433 ms / 0.0006 s / 0.0000 min] upsilon
[00001.9569 ms / 0.0020 s / 0.0000 min] parse
bdd size = 4 with satcount = 768.0
=== COMPUTING BLAME for x1 in BDD with size 4 ===
k=0 size ell=3 d result=0.5000 max increase possible=0.2500 
k=1 size ell=4 d result=0.1250 max increase possible=0.0833 
k=2 
stopped earlier because no change in g(f,0,k) and g(f,1,k) occurred.
=== DONE ===
[00000.5155 ms / 0.0005 s / 0.0000 min] blame
[00000

Todo test cases

* Compare importance values from thesis with those that are computed here...
* Shapley value?